# ⚙️ Notebook 2: Feature Engineering & Preprocessing
## Residual Physics-Aware Neural ODE for Flight Attitude Prediction

**Input from Notebook 1:**
- 100 trajectories
- 7.7M timesteps  
- 8 aircraft types

**This Notebook:**
1. Load data from Notebook 1
2. Engineer physics features
3. Compute derivatives  
4. Normalize data
5. Create sequences
6. Split train/val/test
7. Export for training

---
## 1. Setup & Load Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from pathlib import Path
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline
print("✓ Libraries imported")

✓ Libraries imported


In [2]:
# Load from Notebook 1
print("Loading data from Notebook 1...")

with open('./data/raw/opensky_trajectories.pkl', 'rb') as f:
    trajectories = pickle.load(f)

df_full = pd.read_csv('./data/raw/opensky_flight_data.csv')

with open('./data/raw/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

print(f"✓ Loaded {len(trajectories)} trajectories")
print(f"✓ DataFrame: {df_full.shape}")
print(f"✓ Timesteps: {len(df_full):,}")
print(f"✓ Sampling rate: {metadata['sampling_rate']} sec")

Loading data from Notebook 1...
✓ Loaded 100 trajectories
✓ DataFrame: (7684932, 18)
✓ Timesteps: 7,684,932
✓ Sampling rate: 0.1 sec


---
## 2. Physics Feature Engineering

In [3]:
# Aircraft parameters (scaled by type)
AIRCRAFT_PARAMS = {
    'mass': 1200.0,
    'Ixx': 1285.0,
    'Iyy': 3200.0,
    'Izz': 4160.0,
    'rho': 1.225,
    'g': 9.81
}

AIRCRAFT_SCALE = {
    'B738': 2.5, 'B737': 2.5, 'A320': 2.3,
    'A321': 2.6, 'B77W': 5.0, 'A319': 2.2,
    'B763': 3.5, 'A333': 4.0
}

print("✓ Aircraft parameters defined")

✓ Aircraft parameters defined


In [4]:
def add_physics_features(df, aircraft_type='B738'):
    """Add energy, momentum, Euler angles, dynamic pressure"""
    scale = AIRCRAFT_SCALE.get(aircraft_type, 2.5)
    mass = AIRCRAFT_PARAMS['mass'] * scale
    Ixx = AIRCRAFT_PARAMS['Ixx'] * scale**2
    Iyy = AIRCRAFT_PARAMS['Iyy'] * scale**2
    Izz = AIRCRAFT_PARAMS['Izz'] * scale**2
    
    I_matrix = np.diag([Ixx, Iyy, Izz])
    omega = df[['p', 'q', 'r']].values
    
    # Energy
    E_rot = np.array([0.5 * omega[i] @ I_matrix @ omega[i] for i in range(len(omega))])
    E_trans = 0.5 * mass * df['V']**2
    
    # Angular momentum
    L = np.array([I_matrix @ omega[i] for i in range(len(omega))])
    
    # Euler angles from quaternion
    q0, q1, q2, q3 = df['q0'], df['q1'], df['q2'], df['q3']
    phi = np.arctan2(2*(q0*q1 + q2*q3), 1 - 2*(q1**2 + q2**2))
    theta = np.arcsin(np.clip(2*(q0*q2 - q3*q1), -1, 1))
    psi = np.arctan2(2*(q0*q3 + q1*q2), 1 - 2*(q2**2 + q3**2))
    
    # Dynamic pressure
    q_dyn = 0.5 * AIRCRAFT_PARAMS['rho'] * df['V']**2
    
    # Add to dataframe
    df['E_rot'] = E_rot
    df['E_trans'] = E_trans
    df['E_total'] = E_rot + E_trans
    df['Lx'] = L[:, 0]
    df['Ly'] = L[:, 1]
    df['Lz'] = L[:, 2]
    df['L_mag'] = np.linalg.norm(L, axis=1)
    df['phi'] = phi
    df['theta'] = theta
    df['psi'] = psi
    df['q_dyn'] = q_dyn
    
    return df

print("✓ Feature engineering function defined")

✓ Feature engineering function defined


In [5]:
# Apply to all trajectories
print("Engineering features for 7.7M timesteps...")
print("This will take 2-3 minutes...\n")

df_list = []
for traj_id in df_full['trajectory_id'].unique():
    df_traj = df_full[df_full['trajectory_id'] == traj_id].copy()
    aircraft = df_traj['aircraft_type'].iloc[0]
    df_traj = add_physics_features(df_traj, aircraft)
    df_list.append(df_traj)
    
    if (traj_id + 1) % 10 == 0:
        print(f"  {traj_id + 1}/100")

df_features = pd.concat(df_list, ignore_index=True)
print(f"\n✓ Added {df_features.shape[1] - df_full.shape[1]} features")
print(f"  Total columns: {df_features.shape[1]}")

Engineering features for 7.7M timesteps...
This will take 2-3 minutes...

  10/100
  20/100
  30/100
  40/100
  50/100
  60/100
  70/100
  80/100
  90/100
  100/100

✓ Added 11 features
  Total columns: 29


---
## 3. Compute Derivatives

In [6]:
def compute_derivatives(df_traj, dt=0.1):
    """Compute state derivatives for physics loss"""
    state_cols = ['q0', 'q1', 'q2', 'q3', 'p', 'q', 'r', 'V', 'alpha', 'beta']
    for col in state_cols:
        df_traj[f'{col}_dot'] = np.gradient(df_traj[col].values, dt)
    return df_traj

print("Computing derivatives...")
df_list = []
for traj_id in df_features['trajectory_id'].unique():
    df_traj = df_features[df_features['trajectory_id'] == traj_id].copy()
    df_traj = compute_derivatives(df_traj, metadata['sampling_rate'])
    df_list.append(df_traj)
    if (traj_id + 1) % 10 == 0:
        print(f"  {traj_id + 1}/100")

df_with_derivs = pd.concat(df_list, ignore_index=True)
print(f"\n✓ Derivatives computed")

Computing derivatives...
  10/100
  20/100
  30/100
  40/100
  50/100
  60/100
  70/100
  80/100
  90/100
  100/100

✓ Derivatives computed


---
## 4. Normalization

In [7]:
STATE_COLS = ['q0', 'q1', 'q2', 'q3', 'p', 'q', 'r', 'V', 'alpha', 'beta']
CONTROL_COLS = ['elevator', 'aileron', 'rudder', 'throttle']
FEATURE_COLS = ['E_total', 'E_rot', 'E_trans', 'L_mag', 'Lx', 'Ly', 'Lz', 'phi', 'theta', 'psi', 'q_dyn']
DERIVATIVE_COLS = [f'{c}_dot' for c in STATE_COLS]

# Use RobustScaler for real data (handles outliers)
state_scaler = RobustScaler()
control_scaler = MinMaxScaler(feature_range=(-1, 1))
feature_scaler = RobustScaler()
derivative_scaler = RobustScaler()

df_scaled = df_with_derivs.copy()
df_scaled[STATE_COLS] = state_scaler.fit_transform(df_with_derivs[STATE_COLS])
df_scaled[CONTROL_COLS] = control_scaler.fit_transform(df_with_derivs[CONTROL_COLS])
df_scaled[FEATURE_COLS] = feature_scaler.fit_transform(df_with_derivs[FEATURE_COLS])
df_scaled[DERIVATIVE_COLS] = derivative_scaler.fit_transform(df_with_derivs[DERIVATIVE_COLS])

print("✓ Data scaled")

# Save scalers
with open('./data/processed/scalers.pkl', 'wb') as f:
    pickle.dump({
        'state': state_scaler,
        'control': control_scaler,
        'feature': feature_scaler,
        'derivative': derivative_scaler
    }, f)
print("✓ Scalers saved")

✓ Data scaled
✓ Scalers saved


---
## 5. Create Sequences

In [8]:
def create_sequences(df, traj_ids, seq_len=500, stride=250):
    """Create overlapping sequences"""
    sequences = []
    for traj_id in traj_ids:
        df_traj = df[df['trajectory_id'] == traj_id].reset_index(drop=True)
        if len(df_traj) < seq_len:
            continue
        
        for start in range(0, len(df_traj) - seq_len + 1, stride):
            end = start + seq_len
            sequences.append({
                'states': df_traj.loc[start:end-1, STATE_COLS].values,
                'controls': df_traj.loc[start:end-1, CONTROL_COLS].values,
                'features': df_traj.loc[start:end-1, FEATURE_COLS].values,
                'derivatives': df_traj.loc[start:end-1, DERIVATIVE_COLS].values,
                'times': df_traj.loc[start:end-1, 'time'].values,
                'traj_id': traj_id,
                'aircraft': df_traj['aircraft_type'].iloc[0]
            })
    return sequences

all_ids = df_scaled['trajectory_id'].unique()
all_seqs = create_sequences(df_scaled, all_ids, 500, 250)
print(f"✓ Created {len(all_seqs)} sequences")
print(f"  Length: 500 steps (50 sec)")
print(f"  Overlap: 50%")

✓ Created 30586 sequences
  Length: 500 steps (50 sec)
  Overlap: 50%


---
## 6. Train/Val/Test Split

In [9]:
# Split by trajectory to avoid leakage
train_ids, temp_ids = train_test_split(list(all_ids), test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

train_seqs = [s for s in all_seqs if s['traj_id'] in train_ids]
val_seqs = [s for s in all_seqs if s['traj_id'] in val_ids]
test_seqs = [s for s in all_seqs if s['traj_id'] in test_ids]

print(f"Sequences:")
print(f"  Train: {len(train_seqs)}")
print(f"  Val: {len(val_seqs)}")
print(f"  Test: {len(test_seqs)}")

Sequences:
  Train: 21779
  Val: 4365
  Test: 4442


---
## 7. Export

In [10]:
with open('./data/processed/train_sequences.pkl', 'wb') as f:
    pickle.dump(train_seqs, f)
with open('./data/processed/val_sequences.pkl', 'wb') as f:
    pickle.dump(val_seqs, f)
with open('./data/processed/test_sequences.pkl', 'wb') as f:
    pickle.dump(test_seqs, f)

metadata_prep = {
    'state_cols': STATE_COLS,
    'control_cols': CONTROL_COLS,
    'feature_cols': FEATURE_COLS,
    'derivative_cols': DERIVATIVE_COLS,
    'train_ids': train_ids,
    'val_ids': val_ids,
    'test_ids': test_ids,
    'seq_len': 500,
    'sampling_rate': 0.1
}

with open('./data/processed/metadata.pkl', 'wb') as f:
    pickle.dump(metadata_prep, f)

print("✓ All data saved")
print("\n" + "="*60)
print("NOTEBOOK 2 COMPLETE")
print("="*60)
print("\nReady for Notebook 3: Model Training")

✓ All data saved

NOTEBOOK 2 COMPLETE

Ready for Notebook 3: Model Training
